In [33]:
from __init__ import *
from snmcseq_utils import create_logger
from CEMBA_update_mysql import connect_sql


In [34]:
log = create_logger()
log.info("Hello")

03/04/2018 07:56:27 PM Hello


In [36]:
ens = 'Ens1'
cluster_type = 'cluster_mCHmCG_lv_npc50_k30'
annotation_type = 'annotation' + cluster_type[len('cluster'):]
engine = connect_sql('CEMBA')

# cluster annotation
sql = """SELECT {}, {} FROM {}""".format(cluster_type, annotation_type, ens)
df_info = pd.read_sql(sql, engine)
df_info = df_info.sort_values(cluster_type).drop_duplicates().set_index(cluster_type)

ens_path = os.path.join(PATH_ENSEMBLES, ens)
n_clusters = len(glob.glob(os.path.join(ens_path, 'allc_merged/allc_merged_mCG_{}_*_{}.tsv'.format(cluster_type, ens))))

# dmr results
input_f = os.path.join(ens_path, 'dmr/dmr_allc_merged_mCG_{}_rms_results_collapsed.tsv'.format(cluster_type))
df = pd.read_table(input_f, index_col=['#chr', 'start', 'end'], dtype={'#chr': object})

In [37]:
def dmr_counts(ens, n_clusters, cluster_type, df, keys=['hypomethylated_samples', 'hypermethylated_samples']):
    """
    cluster_type examples:
    cluster_type = 'cluster_mCHmCG_lv_npc50_k30'
    
    """
    res_out = []
    for key in keys:
        res_key = []
        res = df.loc[~df[key].isnull(), key].apply(lambda x: x.split(','))
        for i in range(n_clusters):
            res_cluster = res.loc[
                res.apply(lambda x: ('merged_mCG_{}_{}_{}'.format(cluster_type, i+1, ens) in x))]
            res_key.append({cluster_type: i+1, 'n_dmr_{}'.format(key[:4]): res_cluster.shape[0]})
        res_key = pd.DataFrame(res_key).set_index(cluster_type)
        res_out.append(res_key)
    return pd.concat(res_out, axis=1)

In [38]:
df_ndmr = dmr_counts(ens, n_clusters, cluster_type, df)
df_ndmr = pd.merge(df_info, df_ndmr, left_index=True, right_index=True)

In [39]:
df_ndmr.head()

,annotation_mCHmCG_lv_npc50_k30,n_dmr_hypo,n_dmr_hype
cluster_mCHmCG_lv_npc50_k30,,,
1,mL4,41566,6619
2,mL2/3,57687,5425
3,mL2/3,32924,1547
4,mL5-1,16510,1874
5,mL6-2,29221,5771


In [41]:
print(df.shape)
df_hypo = df.loc[(df['number_of_dms']>=3) & (~df['hypomethylated_samples'].isnull()), 'hypomethylated_samples'].apply(lambda x: x.split(','))
df_hyper = df.loc[(df['number_of_dms']>=3) & (~df['hypermethylated_samples'].isnull()), 'hypermethylated_samples'].apply(lambda x: x.split(','))

print(df_hypo.shape)
print(df_hyper.shape)

(392153, 20)
(60718,)
(44613,)


In [42]:
df_hypo.head()

#chr  start    end    
1     3391591  3391804      [merged_mCG_cluster_mCHmCG_lv_npc50_k30_1_Ens1]
      3498321  3498399    [merged_mCG_cluster_mCHmCG_lv_npc50_k30_5_Ens1...
      3637646  3637688      [merged_mCG_cluster_mCHmCG_lv_npc50_k30_4_Ens1]
      4471128  4471240      [merged_mCG_cluster_mCHmCG_lv_npc50_k30_2_Ens1]
      4688593  4688647    [merged_mCG_cluster_mCHmCG_lv_npc50_k30_4_Ens1...
Name: hypomethylated_samples, dtype: object

In [45]:
if not os.path.isdir('/scratch/dmr'):
    os.makedirs('/scratch/dmr')
    logging.info('Created directory: /scratch/dmr')
    
for i in range(n_clusters):
    df_hypo_cluster = df_hypo.loc[
        df_hypo.apply(lambda x: ('merged_mCG_{}_{}_{}'.format(cluster_type, i+1, ens) in x))]
    output = os.path.join('/scratch/dmr', 'dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst{}.bed'.format(i+1))
    df_hypo_cluster.to_csv(output, sep='\t', header=False, index=True, na_rep='NA')
    logging.info("Saved to {}".format(output))
    logging.info(df_hypo_cluster.shape)

03/04/2018 08:24:43 PM Created directory: /scratch/dmr
03/04/2018 08:24:43 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst1.bed
03/04/2018 08:24:43 PM (16977,)
03/04/2018 08:24:44 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst2.bed
03/04/2018 08:24:44 PM (19580,)
03/04/2018 08:24:45 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst3.bed
03/04/2018 08:24:45 PM (14284,)
03/04/2018 08:24:45 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst4.bed
03/04/2018 08:24:45 PM (6308,)
03/04/2018 08:24:46 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst5.bed
03/04/2018 08:24:46 PM (11274,)
03/04/2018 08:24:46 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst6.bed
03/04/2018 08:24:46 PM (7308,)
03/04/2018 08:24:47 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst7.bed